In [51]:
# importing numpy and pandas
import numpy as np
import pandas as pd

In [52]:
# first we are working on user id data

user_data = open("user.dat","r").read()

In [53]:
# the file contain non ACSII and symboles that we dont need we are only keeping the data with numbers and char

a = ""
for i in user_data:
    if i.isalnum():
        a += i
    else:
        a += " "
a = a.split()
name = [a[i] for i in range(len(a)) if i%2==0]
user_id = [int(a[i]) for i in range(len(a)) if i%2==1]
user = dict(zip(user_id,name))
user_df = pd.DataFrame(user.values(),index=user.keys())
user_df.reset_index(inplace=True)
user_df.rename(columns={0:"Name","index":"id"},inplace=True)
user_df  # our work has done we get the data of users in this variable

,id,Name
0,1,Saheba
1,2,Lokesh
2,3,Hemant
3,4,Abhishek
4,5,Kuldeep
5,6,Dev
6,7,Siddharth
7,8,Ajay
8,9,Sachin
9,10,Varun


In [54]:
# this is the main file which has the attendance data and reading it in csv

df = pd.read_csv("attendance.csv",header=None)

In [55]:
# because the data is in one column spliting it and keeping only the req. two columns so make these function that extract the data in first two columns

def get_id(text):
    return text.split("\t")[0]
def get_time(text):
    return text.split("\t")[1]

In [56]:
# creating two columns ID and Time that hold the data when the employee punch in and out

df["id"]= df[0].map(get_id)
df["time"] = df[0].map(get_time)
df.drop(columns=[0],inplace=True)  # deleting the raw data column and keeping only two column

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      443 non-null    object
 1   time    443 non-null    object
dtypes: object(2)
memory usage: 7.1+ KB


In [58]:
df['id']= df["id"].astype(int)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      443 non-null    int64 
 1   time    443 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.1+ KB


In [60]:
data = pd.merge(df,user_df,how="inner",on="id")

In [61]:
data

,id,time,Name
0,8,2024-12-02 08:39:37,Ajay
1,9,2024-12-02 09:04:17,Sachin
2,23,2024-12-02 09:17:46,Devesh
3,25,2024-12-02 09:33:58,Arvind
4,24,2024-12-02 09:54:28,Purnima
...,...,...,...
438,26,2024-12-31 10:29:05,Diksha
439,4,2024-12-31 10:44:18,Abhishek
440,1,2024-12-31 10:50:33,Saheba
441,5,2024-12-31 10:59:04,Kuldeep


In [62]:
data['datetime'] = pd.to_datetime(df['time'])
data['date'] = pd.to_datetime(df['time']).dt.date
data['in_out_time'] = pd.to_datetime(df['time']).dt.time
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           443 non-null    int64         
 1   time         443 non-null    object        
 2   Name         443 non-null    object        
 3   datetime     443 non-null    datetime64[ns]
 4   date         443 non-null    object        
 5   in_out_time  443 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 20.9+ KB


In [63]:
data["am_pm"] = data['datetime'].dt.strftime("%p")
data_copy = data.copy()

In [64]:
data.drop(columns="time",inplace=True)

In [65]:
ampm = data.pivot_table(index=data.index,columns='am_pm',values='datetime',aggfunc="min")
ampm.reset_index(inplace=True)
ampm.rename(columns={"index":"row_id"},inplace=True)
ampm

am_pm,row_id,AM,PM
0,0,2024-12-02 08:39:37,NaT
1,1,2024-12-02 09:04:17,NaT
2,2,2024-12-02 09:17:46,NaT
3,3,2024-12-02 09:33:58,NaT
4,4,2024-12-02 09:54:28,NaT
...,...,...,...
438,438,2024-12-31 10:29:05,NaT
439,439,2024-12-31 10:44:18,NaT
440,440,2024-12-31 10:50:33,NaT
441,441,2024-12-31 10:59:04,NaT


In [66]:
data = data.reset_index().rename(columns={"index":"row_id"})

In [67]:
data = pd.merge(data,ampm,how='inner',on ='row_id')

In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443 entries, 0 to 442
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   row_id       443 non-null    int64         
 1   id           443 non-null    int64         
 2   Name         443 non-null    object        
 3   datetime     443 non-null    datetime64[ns]
 4   date         443 non-null    object        
 5   in_out_time  443 non-null    object        
 6   am_pm        443 non-null    object        
 7   AM           236 non-null    datetime64[ns]
 8   PM           207 non-null    datetime64[ns]
dtypes: datetime64[ns](3), int64(2), object(4)
memory usage: 31.3+ KB


In [88]:
timesheet = pd.read_excel("timesheet.xlsx",sheet_name="time")

In [103]:
# pd.to_datetime(timesheet['max late']).dt.time
ts = timesheet[['Name','max late','max early']]

In [104]:
timesheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Name       14 non-null     object        
 1   In time    14 non-null     datetime64[ns]
 2   Out time   14 non-null     datetime64[ns]
 3   max late   14 non-null     datetime64[ns]
 4   max early  14 non-null     datetime64[ns]
dtypes: datetime64[ns](4), object(1)
memory usage: 692.0+ bytes


In [110]:
data = pd.merge(data,ts,how="inner",on ='Name')

In [115]:
pd.DataFrame(data.groupby(["Name","date","am_pm"])['datetime'].min())

datetime
Name     date       am_pm                    
Abhishek 2024-12-03 AM    2024-12-03 10:52:22
         2024-12-04 AM    2024-12-04 10:57:25
                    PM    2024-12-04 20:30:29
         2024-12-05 AM    2024-12-05 10:48:01
         2024-12-06 AM    2024-12-06 10:52:44
...                                       ...
Santosh  2024-12-28 AM    2024-12-28 09:05:07
                    PM    2024-12-28 18:07:16
         2024-12-30 AM    2024-12-30 09:02:54
                    PM    2024-12-30 18:11:11
         2024-12-31 AM    2024-12-31 09:00:53

[418 rows x 1 columns]

In [121]:
data['AM']>data['max late']

0      False
1      False
2      False
3      False
4      False
       ...  
438    False
439    False
440    False
441    False
442    False
Length: 443, dtype: bool

In [126]:
data['in_out_time']>data['max late'].dt.time

0      False
1      False
2      False
3      False
4      False
       ...  
438    False
439    False
440    False
441    False
442     True
Length: 443, dtype: bool

In [128]:
data.head(3).T

,0,1,2
row_id,0,1,2
id,8,9,23
Name,Ajay,Sachin,Devesh
datetime,2024-12-02 08:39:37,2024-12-02 09:04:17,2024-12-02 09:17:46
date,2024-12-02,2024-12-02,2024-12-02
in_out_time,08:39:37,09:04:17,09:17:46
am_pm,AM,AM,AM
AM,2024-12-02 08:39:37,2024-12-02 09:04:17,2024-12-02 09:17:46
PM,NaT,NaT,NaT
max late,2025-01-07 09:10:00,2025-01-07 09:10:00,2025-01-07 09:40:00


In [132]:
data['max late'] = data['max late'].dt.time
data['max early'] = data['max early'].dt.time
data['AM'] = data['AM'].dt.time
data['PM'] = data['PM'].dt.time
data

,row_id,id,Name,datetime,date,in_out_time,am_pm,AM,PM,max late,max early
0,0,8,Ajay,2024-12-02 08:39:37,2024-12-02,08:39:37,AM,08:39:37,NaT,09:10:00,17:50:00
1,1,9,Sachin,2024-12-02 09:04:17,2024-12-02,09:04:17,AM,09:04:17,NaT,09:10:00,17:50:00
2,2,23,Devesh,2024-12-02 09:17:46,2024-12-02,09:17:46,AM,09:17:46,NaT,09:40:00,17:50:00
3,3,25,Arvind,2024-12-02 09:33:58,2024-12-02,09:33:58,AM,09:33:58,NaT,09:40:00,17:50:00
4,4,24,Purnima,2024-12-02 09:54:28,2024-12-02,09:54:28,AM,09:54:28,NaT,10:10:00,18:20:00
...,...,...,...,...,...,...,...,...,...,...,...
438,438,26,Diksha,2024-12-31 10:29:05,2024-12-31,10:29:05,AM,10:29:05,NaT,10:40:00,18:20:00
439,439,4,Abhishek,2024-12-31 10:44:18,2024-12-31,10:44:18,AM,10:44:18,NaT,11:10:00,20:20:00
440,440,1,Saheba,2024-12-31 10:50:33,2024-12-31,10:50:33,AM,10:50:33,NaT,11:10:00,18:20:00
441,441,5,Kuldeep,2024-12-31 10:59:04,2024-12-31,10:59:04,AM,10:59:04,NaT,11:10:00,17:50:00


In [134]:
# filling null values in AM column with max late and max early
data['AM'] = data['AM'].fillna(data['max late'])
data['PM'] = data['PM'].fillna(data['max early'])

In [137]:
data_copy_2 = data.copy()

In [141]:
data['late_arival'] = data['AM']>data['max late']
data['early leave'] = data['PM']<data['max early']

In [147]:
late_data = data.groupby("Name").agg({"late_arival":"sum","early leave":"sum"})

In [165]:
# ta stand for total attendance a temprary variable 
ta = pd.DataFrame(data.groupby(["Name",'date'])['date'].count()).rename({"date":"count"},axis=1).reset_index()

In [168]:
total_present = pd.DataFrame(ta.groupby("Name")['date'].count())

In [169]:
total_present

,date
Name,
Abhishek,24
Aditi,3
Ajay,8
Arvind,25
Bantu,23
Dev,9
Devesh,21
Diksha,24
Kuldeep,24


In [170]:
late_data

,late_arival,early leave
Name,,
Abhishek,0,0
Aditi,0,1
Ajay,3,0
Arvind,0,0
Bantu,3,1
Dev,8,2
Devesh,0,1
Diksha,0,1
Kuldeep,0,0


In [173]:
final = pd.merge(total_present,late_data,on='Name',how='inner').reset_index()

In [174]:
final

,Name,date,late_arival,early leave
0,Abhishek,24,0,0
1,Aditi,3,0,1
2,Ajay,8,3,0
3,Arvind,25,0,0
4,Bantu,23,3,1
5,Dev,9,8,2
6,Devesh,21,0,1
7,Diksha,24,0,1
8,Kuldeep,24,0,0
9,Purnima,22,0,1


In [178]:
final['net_present_day']=final['date']-(final['late_arival']*0.5 + final['early leave']*0.5)

In [179]:
final

,Name,date,late_arival,early leave,net_present_day
0,Abhishek,24,0,0,24.0
1,Aditi,3,0,1,2.5
2,Ajay,8,3,0,6.5
3,Arvind,25,0,0,25.0
4,Bantu,23,3,1,21.0
5,Dev,9,8,2,4.0
6,Devesh,21,0,1,20.5
7,Diksha,24,0,1,23.5
8,Kuldeep,24,0,0,24.0
9,Purnima,22,0,1,21.5


In [180]:
final.to_excel("Final report.xlsx")